POBLACION ACTIVA (FUERZA LABORAL)<br>
extraido WDB - "SL.TLF.TOTL.IN" - DB=2

In [1]:
#importo librerias
import wbgapi as wb
import pandas as pd

In [2]:
#Filtro por pais del Continente Americano
paises = ['ATG','ARG','BRB','BLZ','BOL','BRA','CAN','CHL','COL','CRI','CUB','DMA','DOM','ECU','SLV','GRD','GTM','GUY','HTI','HND','JAM','MEX','NIC','PAN','PRY','PER','KNA','LCA','VCT','SUR','BHS','TTO','USA','URY','VEN']

In [3]:
#POBLACION ACTIVA (FUERZA LABORAL) - extraido de WDB - "SL.TLF.TOTL.IN" - DB=2
#CREACION DE DATAFRAME
tbl_fuerza_laboral = pd.read_csv('Extraccion/tbl_fuerza_laboral.csv')
#tbl_fuerza_laboral.columns=["Pais_ID","Year","Fza_Laboral"]

In [5]:
tbl_fuerza_laboral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_pais          735 non-null    object 
 1   year             735 non-null    int64  
 2    fuerza_laboral  651 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 17.4+ KB


In [4]:
#CONTAMOS VALORES NULOS
tbl_fuerza_laboral.isnull().sum()

id_pais             0
year                0
 fuerza_laboral    84
dtype: int64

HaY 84 datos faltantes, algunos paises tienen todos sus datos faltantes<br>
Vamos a identificarlos

In [5]:
#Vemos que paises tienen TODOS sus datos faltantes
print("Paises con TODOS sus datos nulos")
paises_nulos=[]
for i in paises:
    mask = tbl_fuerza_laboral["id_pais"] == i
    df_filtrada = tbl_fuerza_laboral[mask]
    a = df_filtrada.iloc[:,2].isnull().sum()
    if a == 21:
        paises_nulos.append(i)
print(paises_nulos)

Paises con TODOS sus datos nulos
['ATG', 'DMA', 'GRD', 'KNA']


LOS 84 DATOS FALTANTES CORRESPONDEN A CUATROS PAISES:<br>
ATG, DMA, GRD, KNA<br>
VAMOS A LLENARLOS POR MEDIO DE COMPARACION CON PAISES SIMILARES<br>
ADEMAS, VAMOS A USAR LA TABLA DE POBLACION

ATG, DMA, GRD y KNA son paises muy similares entre si. <br>
Paises pequeños del caribe, con poca poblacion. Sus PIB son similares y estan ranqueados entre los últimos paises por su volumen de PIB.<br>
El indice de desarrollo humano los hubica en las posiciones, 78, 79, 74 y 75, respectivamente.<br>
Usaremos a LCA como referencia para completar los valores faltantes<br>

FUENTE: https://datosmacro.expansion.com/

vamos a obtener los datos de las poblaciones de los paises en cuestion

In [6]:
#POBLACION - extraido de WDB - "SP.POP.TOTL" - DB=2
wb.db= 2
pob = wb.data.DataFrame('SP.POP.TOTL', ["ATG","DMA","GRD","KNA","LCA"], time=range(2000, 2021), labels=True, columns="economy").reset_index()
pob.drop(["time"], axis=1, inplace=True)
pob

,Time,ATG,DMA,GRD,KNA,LCA
0,2020,97928.0,71991.0,112519.0,53192.0,183629.0
1,2019,97115.0,71808.0,112002.0,52834.0,182795.0
2,2018,96282.0,71626.0,111449.0,52438.0,181890.0
3,2017,95425.0,71460.0,110874.0,52036.0,180955.0
4,2016,94520.0,71307.0,110263.0,51629.0,180028.0
5,2015,93571.0,71175.0,109603.0,51204.0,179131.0
6,2014,92562.0,71091.0,108900.0,50776.0,178307.0
7,2013,91510.0,71019.0,108172.0,50328.0,177505.0
8,2012,90407.0,70954.0,107452.0,49881.0,176654.0
9,2011,89250.0,70912.0,106786.0,49442.0,175538.0


CALCULAREMOS QUE PORCENTAJE DE LA POBLACION DE "LCA" CORRESPONDE A LA FUERZA LABORAL DEL PAIS Y USAREMOS ESTE DATO PARA COMPLETAR LA DEL RESTO DE LOS PAISES

In [8]:
tbl_fuerza_laboral.head(1)

,id_pais,year,fuerza_laboral
0,VEN,2020,10459875.0


In [11]:
#Creamos un data frame con la Fuerza Laboral de LCA
df1=(tbl_fuerza_laboral[tbl_fuerza_laboral["id_pais"] == "LCA"]).reset_index()

In [12]:
df1

,index,id_pais,year,fuerza_laboral
0,147,LCA,2020,102251.0
1,148,LCA,2019,103907.0
2,149,LCA,2018,104621.0
3,150,LCA,2017,103128.0
4,151,LCA,2016,101697.0
5,152,LCA,2015,100273.0
6,153,LCA,2014,98715.0
7,154,LCA,2013,97118.0
8,155,LCA,2012,95508.0
9,156,LCA,2011,91341.0


In [17]:
#Creamos otro DataFrame que divida la fuerza laboral de LCA por su poblacion
df2 = round((df1[' fuerza_laboral'] / pob["LCA"]),2)

Cargamos los datos al DataFrame, vamos a multiplicar a la poblacion de cada país (por cada año) por el factor extraido en el paso anterior

In [24]:
for i in paises_nulos:
    df_prov=pob[i]*df2
    year=2020
    for e in range(0, len(df_prov)):
        mask1= (tbl_fuerza_laboral["id_pais"] == i) & (tbl_fuerza_laboral["year"] == year)
        tbl_fuerza_laboral.loc[mask1, ' fuerza_laboral'] = df_prov.values[e]
        year= int(year)
        year = year-1

In [25]:
#Imprimimos las tablas para ver el resultado
for i in paises_nulos:
    print(tbl_fuerza_laboral[tbl_fuerza_laboral["id_pais"] == i])

    id_pais  year   fuerza_laboral
714     ATG  2020         54839.68
715     ATG  2019         55355.55
716     ATG  2018         55843.56
717     ATG  2017         54392.25
718     ATG  2016         52931.20
719     ATG  2015         52399.76
720     ATG  2014         50909.10
721     ATG  2013         50330.50
722     ATG  2012         48819.78
723     ATG  2011         46410.00
724     ATG  2010         44895.30
725     ATG  2009         44238.93
726     ATG  2008         42697.00
727     ATG  2007         41174.21
728     ATG  2006         39703.20
729     ATG  2005         38287.14
730     ATG  2004         36959.62
731     ATG  2003         36483.06
732     ATG  2002         35234.10
733     ATG  2001         33973.28
734     ATG  2000         32683.01
    id_pais  year   fuerza_laboral
483     DMA  2020         40314.96
484     DMA  2019         40930.56
485     DMA  2018         41543.08
486     DMA  2017         40732.20
487     DMA  2016         39931.92
488     DMA  2015   

In [26]:
#Corroboramos valores nulos
tbl_fuerza_laboral.isnull().sum()

id_pais            0
year               0
 fuerza_laboral    0
dtype: int64